In [17]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.min_rows', 20)

In [18]:
df = pd.read_csv("../data/raw/properties_data.csv")

In [19]:
df.shape

(19980, 52)

In [20]:
# replace values which contains only [.,-] characters with NaN
df['number'].replace(r'^[.,-]+$', np.nan, regex=True, inplace=True)
# remove trailing [.,-] characters
df['number'].replace(r'[.,-]+$', '', regex=True, inplace=True)
# drop duplicate rows
df.drop_duplicates(subset=['latitude', 'longitude', 'price', 'street', 'postalCode', 'number'], inplace=True)

# drop columns
col_to_drop = ['URL',
            'Status',
            'creationDate',
            'lastModificationDate',
            'transactionType',
            'transactionSubtype',
            'country',
            'isSubjectToVat',
            'box',
            'number',
            'street',
            'isFurnished',
            'floor',
            'floorCount',
            'land',
            'toiletCount',
            'epcScore',
            'primaryEnergyConsumptionPerSqm',
            'bookmarkCount',
            'viewCount',
            'locality',
            ]
df.drop(col_to_drop, axis=1, inplace=True)

# drop rows
df.dropna(how='any', subset=['price', 'netHabitableSurface', 'region'], inplace=True)

df.drop(df[(df['type'] == 'APARTMENT_GROUP') | (df['type'] == 'HOUSE_GROUP')].index, inplace=True)

# 'kitchen'
df.loc[(df['kitchen'] == 'HYPER_EQUIPPED') | (df['kitchen'] == 'USA_HYPER_EQUIPPED') | (df['kitchen'] == 'USA_INSTALLED')] = 'SUPER_EQUIPPED'
df.loc[df['kitchen'] == 'USA_SEMI_EQUIPPED'] = 'SEMI_EQUIPPED'
df.loc[df['kitchen'] == 'USA_INSTALLED'] = 'INSTALLED'
df.loc[df['kitchen'] == 'USA_UNINSTALLED'] = 'NOT_INSTALLED'

# cast from float to str
float_col = ['postalCode', 'constructionYear']
df[float_col] = df[float_col].astype(str)

# data imputation
bool_col = ['hasGarden', 'hasTerrace', 'hasSwimmingPool', 'hasAirConditioning', 'hasLift', 'hasDoubleGlazing']
df[bool_col] = df[bool_col].fillna(False)

num_col = ['terraceSurface', 'gardenSurface', 'parkingCountIndoor', 'bathroomCount', 'showerRoomCount', 'bedroomCount']
df[num_col] = df[num_col].fillna(0)

obj_col = ['constructionYear', 'heatingType', 'condition']
df[obj_col] = df[obj_col].fillna('no_info')

df['saleType'] = df['saleType'].fillna('normalSale')

df['subtype'] = df['subtype'].fillna(df['type'])

# cast from float to str
float_col = ['postalCode', 'constructionYear']
df[float_col] = df[float_col].astype(str)

In [22]:
df.shape

(15847, 31)